In [1]:
"""
run 09_do_signal_injection_for_binomial.py

< 240 cpu s per injection trial  per n_inj per GRB per TW

240 cpu s * 100 * 46 * 5 * 6 = 34k CPU hr

34k CPU hr / (230 jobs * 4 CPU) = 37 hr

"""

'\nrun 09_do_signal_injection_for_binomial.py\n\n< 240 cpu s per injection trial  per n_inj per GRB per TW\n\n240 cpu s * 100 * 46 * 5 * 6 = 34k CPU hr\n\n34k CPU hr / (230 jobs * 4 CPU) = 37 hr\n\n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [3]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> 09.txt
# 0: grb_name
# 1: n_inj
# 2: n_trials
##### 3: tw_in_second
# 3: ncpu
# 4: batchIndex
# 5: use_poisson

# Check 1 out of 2

In [5]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/09.sub'
## [::-1] reverse order
grb_names = ['GRB180423A', 'GRB150716A', 'GRB160822A', 'GRB190623A', 'GRB160709B']
n_injs = np.round(np.r_[0:9.1:0.2], 2)
n_trials       = 500
## tws_in_second  = [10, 25, 50, 100, 250, 500]  
ncpu           = 4
batchIndex     = 0
use_poisson    = 1
#######################################

In [6]:
contents = ""
for grb_name in grb_names:       
    for n_inj in n_injs:
        contents += "{}\t".format(grb_name)
        contents += "{}\t".format(n_inj)
        contents += "{}\t".format(n_trials)
        ## contents += "{}\t".format(tw_in_second)
        contents += "{}\t".format(ncpu)
        contents += "{}\t".format(batchIndex)
        contents += "{}\n".format(use_poisson)

with open("09.txt", 'w') as writer:
    writer.write(contents)
    

In [7]:
print(contents)

GRB180423A	0.0	500	4	0	1
GRB180423A	0.2	500	4	0	1
GRB180423A	0.4	500	4	0	1
GRB180423A	0.6	500	4	0	1
GRB180423A	0.8	500	4	0	1
GRB180423A	1.0	500	4	0	1
GRB180423A	1.2	500	4	0	1
GRB180423A	1.4	500	4	0	1
GRB180423A	1.6	500	4	0	1
GRB180423A	1.8	500	4	0	1
GRB180423A	2.0	500	4	0	1
GRB180423A	2.2	500	4	0	1
GRB180423A	2.4	500	4	0	1
GRB180423A	2.6	500	4	0	1
GRB180423A	2.8	500	4	0	1
GRB180423A	3.0	500	4	0	1
GRB180423A	3.2	500	4	0	1
GRB180423A	3.4	500	4	0	1
GRB180423A	3.6	500	4	0	1
GRB180423A	3.8	500	4	0	1
GRB180423A	4.0	500	4	0	1
GRB180423A	4.2	500	4	0	1
GRB180423A	4.4	500	4	0	1
GRB180423A	4.6	500	4	0	1
GRB180423A	4.8	500	4	0	1
GRB180423A	5.0	500	4	0	1
GRB180423A	5.2	500	4	0	1
GRB180423A	5.4	500	4	0	1
GRB180423A	5.6	500	4	0	1
GRB180423A	5.8	500	4	0	1
GRB180423A	6.0	500	4	0	1
GRB180423A	6.2	500	4	0	1
GRB180423A	6.4	500	4	0	1
GRB180423A	6.6	500	4	0	1
GRB180423A	6.8	500	4	0	1
GRB180423A	7.0	500	4	0	1
GRB180423A	7.2	500	4	0	1
GRB180423A	7.4	500	4	0	1
GRB180423A	7.6	500	4	0	1
GRB180423A	7.8	500	4	0	1


In [8]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 230



In [9]:
### create pbs file ---> 09.pbs

# Check 2 out of 2

In [10]:
nameofjob  = "09"
ntotal     = 230           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "09.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 100  # in Hour

In [11]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
############ update ncpu below ############
contents += "#PBS -l nodes=1:ppn={}\n".format(4)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/09.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
############ update args below ############
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "arg3=${paramsArray[3]}\n"
contents += "arg4=${paramsArray[4]}\n"
contents += "arg5=${paramsArray[5]}\n"
############ update script below ############
contents += "python 09_do_signal_injection_for_binomial.py "
############ update arg calls below ############
contents += "--grb_name $arg0 "
contents += "--n_inj $arg1 "
contents += "--n_trials $arg2 "
contents += "--ncpu $arg3 "
contents += "--batchIndex $arg4 "
contents += "--use_poisson $arg5 "
print(contents)
############ update pbs name below ############
with open("09.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 09
#PBS -l nodes=1:ppn=4
#PBS -l pmem=4gb
#PBS -l walltime=100:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/09.out
#PBS -t 1-230%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/09.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
arg3=${paramsArray[3]}
arg4=${paramsArray[4]}
arg5=${paramsArray[5]}
python 09_do_signal_injection_for_binomial.py --grb_name $arg0 --n_inj $arg1 --n_trials $arg2 --ncpu $arg3 --batchIndex $arg4 --use_poisson $arg5 
